In [1]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [3]:
# fetch_top_wallets.py

import requests

def get_top_holders_via_rpc(mint_address: str, helius_api_key: str, limit: int = 60):
    """
    Fetches the largest token accounts for a given SPL mint via Helius RPC.
    Returns up to `limit` entries of {'address': str, 'amount': int}.
    """
    url = f"https://mainnet.helius-rpc.com/?api-key={helius_api_key}"
    
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTokenLargestAccounts",
        "params": [mint_address]
    }

    resp = requests.post(url, json=payload)
    resp.raise_for_status()
    accounts = resp.json()["result"]["value"]
    return accounts[:limit]

if __name__ == "__main__":
    MINT_ADDRESS = "9BB6NFEcjBCtnNLFko2FqVQBq8HHM13kCyYcdQbgpump"
    API_KEY = "148a4123-36e4-43d8-96a0-813bfc274da4"
    LIMIT = 60

    holders = get_top_holders_via_rpc(MINT_ADDRESS, API_KEY, LIMIT)

    print(f"\nTop {len(holders)} Fartcoin Token Accounts:\n")
    for idx, acct in enumerate(holders, start=1):
        print(f"{idx:2d}. Address: {acct['address']}, Amount: {int(acct['amount']):,}")

    # Save addresses to file (optional)
    with open("top_wallets.txt", "w") as f:
        for acct in holders:
            f.write(acct["address"] + "\n")



Top 60 Fartcoin Token Accounts:

 1. Address: ErJLDKQy1Jna9m1LpbLJEXGiEc6yDFxkBu1mXAEZea5o, Amount: 52,327,827,551,741
 2. Address: 7ZFveJ9VZG2AsVBEVjpm8LVbSpv9dXmfQfSoCEtvUMzR, Amount: 48,456,965,438,828
 3. Address: 3YGMNsUBQvFUTFVXdAAc7sEAEKbdDHmf9f4eXyg9uRjd, Amount: 22,852,960,964,631
 4. Address: 7iwuV6jgoKPViaUxDXeny8LoDVe9SqfAJYcCYQKaTJuH, Amount: 17,669,343,113,515
 5. Address: AFR6R3XnxXPwLA7TquXR5hCWqL7TtVchNVTPiYeB1pMx, Amount: 16,147,964,678,150
 6. Address: 8Pow5a6aHqvrZsfYTJdk6fn8U9NBAoP79rckMXiwtVNx, Amount: 13,646,588,139,406
 7. Address: 4zAQ6JTVk9eozjVzAydJPAmK2BYTo8CNDBmoBGxNzRwe, Amount: 13,164,513,425,620
 8. Address: 6sGuMEz4jU4ZisH4SCbTS2zXJW6bRJHtyqo8ASqNSo1P, Amount: 10,297,913,520,693
 9. Address: 76UNncSETpdsCSPzV8BZdbNpdNwYLMVM1sTcmXhybxc3, Amount: 10,001,047,320,067
10. Address: 7bxbfwXi1CY7zWUXW35PBMZjhPD27SarVuHaehMzR2Fn, Amount: 9,604,039,591,002
11. Address: EKFUv1PMw6VfDDgJEa2o8hyKYL6KuiNTHU51nSh8pRzB, Amount: 8,821,909,239,413
12. Address: 3P2VMW2ev

In [5]:
import requests

def register_helius_webhook(api_key, webhook_url, wallet_addresses):
    """
    Registers a webhook with Helius to monitor specified wallet addresses.
    """
    url = f"https://api.helius.xyz/v0/webhooks?api-key={api_key}"

    payload = {
        "webhookURL": webhook_url,
        "transactionTypes": ["TRANSFER"],
        "accountAddresses": wallet_addresses,
        "webhookType": "enhanced",
        "encoding": "jsonParsed"
        # 🔴 Removed "name" as Helius no longer accepts it
    }

    print("\n📤 Sending this payload to Helius:")
    print(payload)

    try:
        response = requests.post(url, json=payload)
        print("🧾 Status Code:", response.status_code)
        print("🧾 Response Text:", response.text)
        response.raise_for_status()
        print("✅ Webhook registered successfully!")
        print("📡 Webhook Info:", response.json())
    except requests.exceptions.HTTPError as err:
        print("❌ Failed to register webhook.")
        print("💥 Error:", err)
    except Exception as e:
        print("❌ Something went wrong.")
        print("💥 Exception:", e)

if __name__ == "__main__":
    API_KEY = "148a4123-36e4-43d8-96a0-813bfc274da4"  # ← Replace with your real key
    WEBHOOK_URL = "https://9af2999b092b.ngrok-free.app/webhook"    # ← Replace with your public ngrok URL

    # Load wallet addresses
    try:
        with open("top_wallets.txt", "r") as file:
            wallet_addresses = [line.strip() for line in file if line.strip()]
    except FileNotFoundError:
        print("❌ Could not find 'top_wallets.txt'. Run fetch_top_wallets.py first.")
        exit()

    if not wallet_addresses:
        print("❌ Wallet address list is empty.")
        exit()

    print(f"\n✅ Loaded {len(wallet_addresses)} wallet addresses.")
    print(wallet_addresses[:5], "...")  # Preview

    # Register webhook
    register_helius_webhook(API_KEY, WEBHOOK_URL, wallet_addresses)



✅ Loaded 60 wallet addresses.
['ErJLDKQy1Jna9m1LpbLJEXGiEc6yDFxkBu1mXAEZea5o', '7ZFveJ9VZG2AsVBEVjpm8LVbSpv9dXmfQfSoCEtvUMzR', '3YGMNsUBQvFUTFVXdAAc7sEAEKbdDHmf9f4eXyg9uRjd', '7iwuV6jgoKPViaUxDXeny8LoDVe9SqfAJYcCYQKaTJuH', 'AFR6R3XnxXPwLA7TquXR5hCWqL7TtVchNVTPiYeB1pMx'] ...

📤 Sending this payload to Helius:
{'webhookURL': 'https://9af2999b092b.ngrok-free.app/webhook', 'transactionTypes': ['TRANSFER'], 'accountAddresses': ['ErJLDKQy1Jna9m1LpbLJEXGiEc6yDFxkBu1mXAEZea5o', '7ZFveJ9VZG2AsVBEVjpm8LVbSpv9dXmfQfSoCEtvUMzR', '3YGMNsUBQvFUTFVXdAAc7sEAEKbdDHmf9f4eXyg9uRjd', '7iwuV6jgoKPViaUxDXeny8LoDVe9SqfAJYcCYQKaTJuH', 'AFR6R3XnxXPwLA7TquXR5hCWqL7TtVchNVTPiYeB1pMx', '8Pow5a6aHqvrZsfYTJdk6fn8U9NBAoP79rckMXiwtVNx', '4zAQ6JTVk9eozjVzAydJPAmK2BYTo8CNDBmoBGxNzRwe', '6sGuMEz4jU4ZisH4SCbTS2zXJW6bRJHtyqo8ASqNSo1P', '76UNncSETpdsCSPzV8BZdbNpdNwYLMVM1sTcmXhybxc3', '7bxbfwXi1CY7zWUXW35PBMZjhPD27SarVuHaehMzR2Fn', 'EKFUv1PMw6VfDDgJEa2o8hyKYL6KuiNTHU51nSh8pRzB', '3P2VMW2evUc4SPE4C8uc6PLPfpRMe9L3mGEbH7NW5Cd